## 1. This section is config and function definitions

In [1]:
import pandas as pd
import plotly.express as px
import numpy as np

titles_map = {
    'avg_increasing_chain_length': 'Average increasing temperature chain', 
    'avg_increasing_chain_length_for_extremes': 'Average increasing temperature chain before extreme rain', 
    'avg_increasing_chain_length_for_extremes_norm': 'Average increasing temperature chain before extreme rain divided by average increasing temperature chain', 
    'avg_high_t_chain_length': 'Average high temperature chain',
    'avg_high_t_chain_length_for_extremes': 'Average high temperature chain before extreme rain',
    'avg_high_t_chain_length_for_extremes_norm': 'Average high temperature chain before extreme rain divided by average high temperature chain',
    'large_jump_pos': 'Rate of large positive temperature jumps', 
    'large_jump_pos_extreme_pp_rate': 'Extreme PP rate for large positive temperature jumps',
    'large_jump_pos_extreme_pp_rate_norm': 'Normalized extreme PP rate for large positive temperature jumps', 
    'large_jump_neg': 'Rate of large negative temperature jumps', 
    'large_jump_neg_extreme_pp_rate': 'Extreme PP rate for large negative temperature jumps',
    'large_jump_neg_extreme_pp_rate_norm': 'Normalized extreme PP rate for large negative temperature jumps',
    'extreme_pp_rate': 'Extreme PP rate',
    'extreme_pp_value': 'Extreme PP value',
    'days_with_pp': 'Number of days with PP',
    'days_total': 'Total number of days'
}

period_map = {
    'historical': 'historical',
    'future': 'ssp585'
}

def run_map(results, 
            metric, 
            size=1, 
            zoom=2, 
            color_scale=px.colors.sequential.RdBu_r, 
            width=800, 
            height=400, 
            opacity=0.5, 
            range_color=(0, 2), 
            land_only=True, 
            model='TaiESM1', 
            period='historical'):
    if land_only:
        map_data = results[results.is_land]
    else:
        map_data = results
    map_data['size'] = size
    
    fig = px.scatter_mapbox(map_data, 
                            lat="lat", 
                            lon="lon",
                            color=metric,
                            size='size',
                            size_max=size,
                            zoom=zoom,
                            color_continuous_scale=color_scale,
                            width=width, 
                            height=height,
                            title=f'{titles_map[metric]} - Model: {model} - Period: {period.capitalize()}',
                            opacity=opacity,
                            range_color=range_color)
    
    fig.update_layout(mapbox_style="open-street-map")
    
    return fig

In [2]:
import glob
import os

folders = os.listdir('results/')

models_available = [(x[0:-len('_historical_Arg')]) for x in folders if x[-len('_historical_Arg'):] == '_historical_Arg']

metrics = ['avg_increasing_chain_length',
 'avg_increasing_chain_length_for_extremes',
 'avg_increasing_chain_length_for_extremes_norm',
 'avg_high_t_chain_length',
 'avg_high_t_chain_length_for_extremes',
 'avg_high_t_chain_length_for_extremes_norm',
 'large_jump_pos',
 'large_jump_pos_extreme_pp_rate',
 'large_jump_neg',
 'large_jump_neg_extreme_pp_rate',
 'large_jump_pos_extreme_pp_rate_norm',
 'large_jump_neg_extreme_pp_rate_norm',
 'extreme_pp_rate',
 'days_with_pp',
 'extreme_pp_rate',
 'extreme_pp_value',
 'days_total'
]

In [3]:
models_available

['era',
 'INM-CM5-0',
 'KIOST-ESM',
 'IPSL-CM6A-LR',
 'ACCESS-ESM1-5',
 'TaiESM1',
 'CanESM5',
 'MIROC6',
 'NorESM2-MM',
 'CNRM-CM6-1']

## 2. This section creates the dictionary with the metrics min and max that are going to be used in the plots

In [7]:
boundaries_map = {m:{'summer':[np.NaN, np.NaN],
                     'winter':[np.NaN, np.NaN]} for m in metrics}

In [8]:
pp_lower_bound = 1
grouping_strategy = 3

for model in models_available:
    for season in ['summer', 'winter']:
        for period in ['historical', 'future']:
            
            viz_results_folder = f'results/{model}_{period_map[period]}_Arg'
            
            if (model != 'era' or period != 'future'):
                results = pd.read_csv(f'./{viz_results_folder}/map_{season}_pp_{pp_lower_bound}_strat_{grouping_strategy}_extremes_local.csv')
            
            for metric in metrics:
                if results.shape[0] > 0 and (model != 'era' or period != 'future'):
                    new_min = min(results[results.is_land][metric].quantile(0.02), boundaries_map[metric][season][0])
                    new_max = max(results[results.is_land][metric].quantile(0.9), boundaries_map[metric][season][1])

                    boundaries_map[metric][season] = (new_min, new_max)

## 3. This section creates the individual contour plots

In [10]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np
import pandas as pd
import seaborn as sns

sns.set_theme(style="whitegrid")
c=0
for season in ['summer', 'winter']:
    for model in models_available:
        for period in ['historical', 'future']:
            for metric in metrics:
                print(c)
                c+=1
                
                if (model != 'era' or period != 'future'):
                    viz_results_folder = f'results/{model}_{period_map[period]}_Arg'
                    results = pd.read_csv(f'./{viz_results_folder}/map_{season}_pp_{pp_lower_bound}_strat_{grouping_strategy}_extremes_local.csv')

                    if results.shape[0] > 0 :
                        # Filtrar datos válidos
                        df = results[~results[metric].isnull()&results.is_land]

                        # Supongamos que el DataFrame tiene columnas 'lon', 'lat', y 'metric'
                        lon = df['lon'].values
                        lat = df['lat'].values
                        value = df[metric].values

                        # Crear grilla (esto asume que los datos están en grilla regular)
                        lon_unique = np.sort(df['lon'].unique())
                        lat_unique = np.sort(df['lat'].unique())
                        lon2d, lat2d = np.meshgrid(lon_unique, lat_unique)

                        # Reorganizar valores a 2D
                        value_2d = df.pivot(index='lat', columns='lon', values=metric).values

                        # Crear figura
                        fig = plt.figure(figsize=(10, 8))
                        ax = plt.axes(projection=ccrs.PlateCarree())

                        range_min = boundaries_map[metric][season][0]
                        range_max = boundaries_map[metric][season][1]

                        levels = np.linspace(range_min, range_max, 100)


                        # Contorno
                        cf = ax.contourf(lon2d, lat2d, value_2d, cmap='RdBu_r', levels=levels,
                                         vmin=range_min, vmax=range_max, extend='both')

                        # Agregar detalles del mapa
                        ax.coastlines()
                        ax.add_feature(cfeature.BORDERS)

                        # Barra de color
                        plt.colorbar(cf, ax=ax, shrink=0.7, label=metric)

                        # Título
                        ax.set_title(f"{model} - {metric} - {period} - {season}", fontsize=12)

                        # Guardar
                        output_path = f"individual_figures/{model}_{metric}_{period}_{season}_contour.png"
                        plt.savefig(output_path, dpi=300)
                        plt.close()


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

# 4.This section creates the panels from the plots that were created in Step 3

In [3]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import glob
import gc

folders = os.listdir('results/')
models_available = [(x[0:-len('_historical_Arg')]) for x in folders if x[-len('_historical_Arg'):] == '_historical_Arg']

metrics = ['avg_increasing_chain_length',
 'avg_increasing_chain_length_for_extremes',
 'avg_increasing_chain_length_for_extremes_norm',
 'avg_high_t_chain_length',
 'avg_high_t_chain_length_for_extremes',
 'avg_high_t_chain_length_for_extremes_norm',
 'large_jump_pos',
 'large_jump_pos_extreme_pp_rate',
 'large_jump_neg',
 'large_jump_neg_extreme_pp_rate',
 'large_jump_pos_extreme_pp_rate_norm',
 'large_jump_neg_extreme_pp_rate_norm',
 'extreme_pp_rate',
 'days_with_pp',
 'extreme_pp_rate',
 'extreme_pp_value',
 'days_total'
]

c=0
for season in ['summer', 'winter']:
    for period in ['historical', 'future']:
        for metric in metrics:
            
            c+=1
            print(c)

            if c>=68:
                # List of uploaded image files
                image_folder =  "individual_figures"
                image_files = [f"{image_folder}/{model}_{metric}_{period}_{season}_contour.png" for model in models_available if (model != 'era' or period != 'future') and model != 'KIOST-ESM']

                print(len(image_files))

                if len(image_files)>9:
                    print(season)
                    print(period)
                    print(metric)

    #sorted([f for f in os.listdir(image_folder) if f.endswith(('png', 'jpg', 'jpeg'))])

    # Panel settings (e.g., 2x2)
                n_rows = 3
                n_cols = 3
                fig, axs = plt.subplots(n_rows, n_cols, figsize=(10, 8))


    # Flatten axes array for easier indexing
                axs = axs.flatten()

    # Plot each image
                for i, img_file in enumerate(image_files):
                    if i >= len(axs): break
                    img = mpimg.imread(img_file)
                    axs[i].imshow(img)
                    axs[i].axis('off')
    #axs[i].set_title(img_file, fontsize=10)

    # Hide unused subplots
                for j in range(i+1, len(axs)):
                    axs[j].axis('off')

                plt.tight_layout()


    # Guardar
                output_path = f"panel_figures/{metric}_{period}_{season}_panel.png"
                plt.savefig(output_path, dpi=300)
                plt.close()
                
               

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
8
